In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import lxml
import sys
import time
import re
import urllib.request


In [2]:
chromedriver_dir = '/Users/mongkyo/Projects/kakao-chatbot/crawling/chromedriver'
# options = webdriver.ChromeOptions()
# options.add_argument('headless')
# options.add_argument('window-size=1920x1080')
# options.add_argument("disable-gpu")
# driver = webdriver.Chrome(chormedriver_dir, chrome_option=options)

## 크롤링 완성

In [31]:
driver = webdriver.Chrome(chromedriver_dir)
driver.get('https://nsulib.nsu.ac.kr/')
elem = driver.find_element_by_id('type1q')  # 학교 홈페이지 검색창
elem.send_keys('전산회계')  # 이 부분에 검색하고 싶은 도서 이름을 넣어주면 된다
elem.send_keys(Keys.RETURN)
a_tag = driver.find_elements_by_xpath('//a[contains(@href,"#previewLocation")]')
for a in a_tag[:3+1]:
    a.click()
time.sleep(1)  # 클릭하고 html코드를 가져오는 과정이 너무 빨라서 클릭한 내용을 적용시키지 못하고 바로 긁어오기 때문에 html코드를 불러올 1초정도의 시간을 주는 것
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
ul = soup.select_one('.resultList')
book_info = []
for li in ul.select('.items'):
    book_one = dict()
    count = 0
    book_one['title'] = li.select_one('dd.title > a').get_text()
    book_one['author'] = li.select('dd.info')[0].get_text()
    book_one['publisher'] = li.select('dd.info')[1].get_text()
    book_one['publish_year'] = li.select('dd.info')[2].get_text()
    book_info.append(book_one)
    tbody = soup.find('tbody')
    for tr in tbody.find_all('tr'):
        book_tmp = dict()
        count += 1
        no = tr.select_one('.footable-first-column').get_text()
        book_tmp['no'] = tr.select_one('.footable-first-column').get_text()
        book_tmp['location'] = tr.select_one('.location').get_text()
        book_tmp['serial_num'] = tr.select_one('.callNum').get_text()
        book_tmp['status'] = tr.select_one('.status').get_text()
        book_tmp['returnDate'] = tr.select_one('.returnDate').get_text()
        book_one[f'{no}'] = book_tmp
    print(f'도서 정보: {book_one}\n 책 권수: {count}')

도서 정보: {'title': '전산회계', 'author': '임재희', 'publisher': '大經', 'publish_year': '2004', '1': {'no': '1', 'location': '중앙도서관/동양서 제1자료실(5층)/', 'serial_num': '325.9 임73ㅈ', 'status': '대출가능', 'returnDate': ''}, '2': {'no': '2', 'location': '중앙도서관/동양서 제1자료실(5층)/', 'serial_num': '325.9 임73ㅈ C.2', 'status': '대출가능', 'returnDate': ''}}
 책 권수: 2
도서 정보: {'title': '전산회계 1.2급 필기', 'author': '공경태', 'publisher': '세학사', 'publish_year': '2005', '1': {'no': '1', 'location': '중앙도서관/동양서 제1자료실(5층)/', 'serial_num': '325.9 임73ㅈ', 'status': '대출가능', 'returnDate': ''}, '2': {'no': '2', 'location': '중앙도서관/동양서 제1자료실(5층)/', 'serial_num': '325.9 임73ㅈ C.2', 'status': '대출가능', 'returnDate': ''}}
 책 권수: 2
도서 정보: {'title': '전산회계', 'author': '김종민', 'publisher': '삼보', 'publish_year': '2007', '1': {'no': '1', 'location': '중앙도서관/동양서 제1자료실(5층)/', 'serial_num': '325.9 임73ㅈ', 'status': '대출가능', 'returnDate': ''}, '2': {'no': '2', 'location': '중앙도서관/동양서 제1자료실(5층)/', 'serial_num': '325.9 임73ㅈ C.2', 'status': '대출가능', 'returnDate': ''}

In [26]:
book_one

{'title': '컴퓨터구조',
 'author': '김동일',
 'publisher': '오성미디어',
 'publish_year': '2002',
 '1': {'no': '1',
  'location': '중앙도서관/동양서 제1자료실(5층)/',
  'serial_num': '028.61 김75ㅋ',
  'status': '대출가능',
  'returnDate': ''},
 '2': {'no': '2',
  'location': '중앙도서관/동양서 제1자료실(5층)/',
  'serial_num': '028.61 김75ㅋ C.2',
  'status': '대출가능',
  'returnDate': ''},
 '3': {'no': '3',
  'location': '중앙도서관/동양서 제1자료실(5층)/',
  'serial_num': '028.61 김75ㅋ C.3',
  'status': '대출가능',
  'returnDate': ''}}

## 아래는 크롤링 과정중에 사용했던 코드들 (지우기 아까워서 냅둠)

### 정적으로 크롤링을 시도

In [20]:
driver = webdriver.Chrome(chromedriver_dir)
driver.get('https://nsulib.nsu.ac.kr/')
elem = driver.find_element_by_name("q")
elem.send_keys(a)
elem.send_keys(Keys.RETURN)
driver.find_element_by_xpath('//*[@id="item_CATTOT000000469834"]/dl/dd[11]/div/p/a').click()
driver.find_element_by_xpath('//*[@id="item_CATTOT000000469834"]/dl/dd[4]/a').click()
info = dict()
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

TypeError: object of type 'WebElement' has no len()

In [ ]:
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
table = soup.find('tbody')


In [114]:
table

<tbody>
<tr><th scope="row">자료유형</th><td>단행본</td></tr><tr><th scope="row">서명/저자사항</th><td>82년생 김지영: 조남주 장편소설 / 조남주 [지음].</td></tr><tr><th scope="row">개인저자</th><td><a href="/search/tot/result?st=FRNT&amp;si=2&amp;q=조남주" title="이 키워드로 더 검색하기">조남주</a>, 저<br/></td></tr><tr><th scope="row">발행사항</th><td>서울 : 민음사, 2016.</td></tr><tr><th scope="row">형태사항</th><td>190 p. ; 20cm.</td></tr><tr><th scope="row">총서사항</th><td><a href="/search/tot/result?st=FRNT&amp;si=1&amp;q=오늘의 젊은 작가;" title="이 키워드로 더 검색하기">오늘의 젊은 작가</a>;13<br/></td></tr><tr><th scope="row">ISBN</th><td>9788937473005(세트)<br/>9788937473135<br/></td></tr><tr><th scope="row">비통제주제어</th><td><a href="/search/tot/result?st=FRNT&amp;si=4&amp;q=한국문학" title="이 키워드로 더 검색하기">한국문학</a>,<a href="/search/tot/result?st=FRNT&amp;si=4&amp;q=한국현대소설" title="이 키워드로 더 검색하기">한국현대소설</a>,<a href="/search/tot/result?st=FRNT&amp;si=4&amp;q=장편소설" title="이 키워드로 더 검색하기">장편소설</a></td></tr><tr><th scope="row">분류기호</th><td><a href="/search/tot/result?st=FRNT&amp;si

In [115]:
for thtd in table.find_all("tr"):
    temp = thtd.th.get_text()
    if temp.replace('\n', '') == '자료유형':
        info['datatype'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == '서명/저자사항':
        info['signature'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == '개인저자':
        info['personal_author'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == '발행사항':
        info['publication_type'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == '형태사항':
        info['form_type'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == '원서명':
        info['original_title'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == 'ISBN':
        info['ISBN'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == '일반주기':
        info['general_note'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == '서지주기':
        info['book_note'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == '일반주제명':
        info['general_subject'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == '분류기호':
        info['class_sign'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == '언어':
        info['language'] = thtd.td.get_text().replace('\n', '')

In [116]:
info

{'datatype': '단행본',
 'signature': '82년생 김지영: 조남주 장편소설 / 조남주 [지음].',
 'personal_author': '조남주, 저',
 'publication_type': '서울 : 민음사, 2016.',
 'form_type': '190 p. ; 20cm.',
 'ISBN': '9788937473005(세트)9788937473135',
 'class_sign': '813.7 ',
 'language': '한국어'}

In [129]:
collect_info = dict()

In [127]:
collect_table = soup.select_one('#divContent > div > div:nth-child(6) > div.searchContents > div > table > tbody')
collect_table.find_all("tr")
for tr in collect_table.find_all("tr"):
    print(tr)
    if driver.find_element_by_class_name("footable-first-column"):
        collect_info['NO'] += tr.td.get_text()

<tr class="first">
<td class="num expand footable-first-column">1</td>
<td class="accessionNo">534762</td>
<td class="callNum">813.7 조1928</td>
<td class="location">
			                    
			                        
			                        
			                        중앙도서관/동양서 제2자료실(6층)/
			                    
	                        </td>
<td>
<span class="status ing">대출중</span>
</td>
<td class="returnDate">2018-12-27</td>
<td>
</td>
<td class="service">
<div class="serviceInfo">
</div>
</td>
<td class="info footable-last-column">
<div class="serviceInfo">
</div>
</td>
</tr>
<tr>
<td class="num expand footable-first-column">2</td>
<td class="accessionNo">534763</td>
<td class="callNum">813.7 조1928 C.2</td>
<td class="location">
			                    
			                        
			                        
			                        중앙도서관/동양서 제2자료실(6층)/
			                    
	                        </td>
<td>
<span class="status available">대출가능</span>
</td>
<td class="return

## 책검색해서 그 페이지에서 책 정보 가져오기(정적) -> (동적으로 고치기)

In [4]:
driver = webdriver.Chrome(chromedriver_dir)
driver.get('https://nsulib.nsu.ac.kr/')
elem = driver.find_element_by_name("q")  # 학교 홈페이지 검색창
elem.send_keys('82년생 김지영')
elem.send_keys(Keys.RETURN)
driver.find_element_by_xpath('//*[@id="item_CATTOT000000011364"]/dl/dd[11]/div/p/a').click()
# html = driver.page_source
# soup = BeautifulSoup(html, 'html.parser')
time.sleep(1)
elem = driver.find_element_by_xpath('//*[@id="holdingW_CAT000000011364_1"]/div[2]')
html = elem.get_attribute('innerHTML')
soup = BeautifulSoup(html, 'html.parser')
tbody = soup.find('tbody')
book_info = []
for tr in tbody.find_all('tr'):
    book_tmp = dict()
    book_tmp['no'] = tr.select_one('.footable-first-column').get_text()
    book_tmp['location'] = tr.select_one('.location').get_text()
    book_tmp['serial_num'] = tr.select_one('.callNum').get_text()
    book_tmp['status'] = tr.select_one('.status').get_text()
    book_tmp['returnDate'] = tr.select_one('.returnDate').get_text()
    book_info.append(book_tmp)
book_info

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="item_CATTOT000000011364"]/dl/dd[11]/div/p/a"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615355 (d5698f682d8b2742017df6c81e0bd8e6a3063189),platform=Mac OS X 10.13.6 x86_64)


## 책 검색 후 여러권을 선택하고나서 책 정보들을 가져오기

In [16]:
a = "컴퓨터구조"

In [22]:
driver = webdriver.Chrome(chromedriver_dir)
driver.get('https://nsulib.nsu.ac.kr/')
elem = driver.find_element_by_id('type1q')  # 학교 홈페이지 검색창
elem.send_keys('82년생 김지영')  # 이 부분에 검색하고 싶은 도서 이름을 넣어주면 된다
elem.send_keys(Keys.RETURN)
a_tag = driver.find_elements_by_xpath('//a[contains(@href,"#previewLocation")]')
for a in a_tag[:3+1]:
    a.click()
time.sleep(1)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
ul = soup.select_one('.resultList')
book_info = []
for li in ul.select('.items'):
    book_one = dict()
    book_one['title'] = li.select_one('dd.title > a').get_text()
    book_one['author'] = li.select('dd.info')[0].get_text()
    book_one['publisher'] = li.select('dd.info')[1].get_text()
    book_one['publish_year'] = li.select('dd.info')[2].get_text()
    book_info.append(book_one)
    tbody = soup.find('tbody')
    for tr in tbody.find_all('tr'):
        book_tmp = dict()
        no = tr.select_one('.footable-first-column').get_text()
        book_tmp['no'] = tr.select_one('.footable-first-column').get_text()
        book_tmp['location'] = tr.select_one('.location').get_text()
        book_tmp['serial_num'] = tr.select_one('.callNum').get_text()
        book_tmp['status'] = tr.select_one('.status').get_text()
        book_tmp['returnDate'] = tr.select_one('.returnDate').get_text()
        book_one[f'{no}'] = book_tmp
    print(book_one)

{'title': '82년생 김지영 : 조남주 장편소설', 'author': '조남주', 'publisher': '민음사', 'publish_year': '2016', '1': {'no': '1', 'location': '중앙도서관/동양서 제2자료실(6층)/', 'serial_num': '813.7 조1928', 'status': '대출중', 'returnDate': '20181227'}, '2': {'no': '2', 'location': '중앙도서관/동양서 제2자료실(6층)/', 'serial_num': '813.7 조1928 C.2', 'status': '대출가능', 'returnDate': ''}, '3': {'no': '3', 'location': '중앙도서관/동양서 제2자료실(6층)/', 'serial_num': '813.7 조1928 C.3', 'status': '대출가능', 'returnDate': ''}}


In [23]:
book_one

{'title': '82년생 김지영 : 조남주 장편소설',
 'author': '조남주',
 'publisher': '민음사',
 'publish_year': '2016',
 '1': {'no': '1',
  'location': '중앙도서관/동양서 제2자료실(6층)/',
  'serial_num': '813.7 조1928',
  'status': '대출중',
  'returnDate': '20181227'},
 '2': {'no': '2',
  'location': '중앙도서관/동양서 제2자료실(6층)/',
  'serial_num': '813.7 조1928 C.2',
  'status': '대출가능',
  'returnDate': ''},
 '3': {'no': '3',
  'location': '중앙도서관/동양서 제2자료실(6층)/',
  'serial_num': '813.7 조1928 C.3',
  'status': '대출가능',
  'returnDate': ''}}

In [10]:
ul = soup.select_one('.resultList')
book_info = []
for li in ul.select('.items'):
    book_one = dict()
    book_one['title'] = li.select_one('dd.title > a').get_text()
    book_one['author'] = li.select('dd.info')[0].get_text()
    book_one['publisher'] = li.select('dd.info')[1].get_text()
    book_one['publish_year'] = li.select('dd.info')[2].get_text()
    book_info.append(book_one)
    tbody = soup.find('tbody')
    if tbody is not None:
        for tr in tbody.find_all('tr'):
            book_tmp = dict()
            no = tr.select_one('.footable-first-column').get_text()
            book_tmp['no'] = tr.select_one('.footable-first-column').get_text()
            book_tmp['location'] = tr.select_one('.location').get_text()
            book_tmp['serial_num'] = tr.select_one('.callNum').get_text()
            book_tmp['status'] = tr.select_one('.status').get_text()
            book_tmp['returnDate'] = tr.select_one('.returnDate').get_text()
            book_one[f'{no}'] = book_tmp
    else:
        
    print(book_one)
#     if 
#     print(li)
#     print('------------------')

{'title': '82년생 김지영 : 조남주 장편소설', 'author': '조남주', 'publisher': '민음사', 'publish_year': '2016'}


In [14]:
soup.select_one('mobileTable')

### dict 자료구조가 익숙하지 않아서 예제를 만들어보면서 구조를 공부함

In [118]:
book1 = {'no': '1', 'location': '중앙도서관', 'date': '2019-01-22'}
book2 = {'no': '2', 'location': '중앙도서관', 'date': ''}

In [125]:
book1['no1'] = book2
book1

{'no': '1',
 'location': '중앙도서관',
 'date': '2019-01-22',
 'no1': {'no': '2', 'location': '중앙도서관', 'date': ''}}

In [ ]:
first_dict = dict()
second_dict = dict()

In [ ]:
second_dict = {'1' : 'first',
              '2': 'second'}

In [46]:
first_dict= second_dict, third_dict

In [49]:
for i in first_dict:
    print(i)

{'1': 'first', '2': 'second'}
{'3': 'three', '4': 'four'}


In [45]:
third_dict = {'3': 'three',
             '4': 'four'}

In [42]:
book_title = dict()
book_reservation = dict()
book_reservation = {'1': '123', '2': '123333333123', '3': '234'}



In [60]:
book_info = {'datatype': '단행본',
 'signature': '82년생 김지영: 조남주 장편소설 / 조남주 [지음].',
 'personal_author': '조남주, 저',
 'publication_type': '서울 : 민음사, 2016.',
 'form_type': '190 p. ; 20cm.',
 'ISBN': '9788937473005(세트)9788937473135',
 'class_sign': '813.7 ',
 'language': '한국어',
            1 : {
                'reservation': '예약가능'
            },
             2 : {
                 'reservation': '예약불가'
             }
}

In [61]:
book_info

{'datatype': '단행본',
 'signature': '82년생 김지영: 조남주 장편소설 / 조남주 [지음].',
 'personal_author': '조남주, 저',
 'publication_type': '서울 : 민음사, 2016.',
 'form_type': '190 p. ; 20cm.',
 'ISBN': '9788937473005(세트)9788937473135',
 'class_sign': '813.7 ',
 'language': '한국어',
 1: {'reservation': '예약가능'},
 2: {'reservation': '예약불가'}}

In [56]:
info = dict
book_info = dict
book_reservation = dict

In [77]:
book_dic = ['첫번째', '두번째', '세번째', '네번째']

In [79]:
book_reset = {book_dic[0]: book_dic[1:]}

In [80]:
book_reset

{'첫번째': ['두번째', '세번째', '네번째']}